In [1]:
import os
import numpy as np
from netCDF4 import Dataset
import time

In [2]:
''' 获取路径下全部文件名 '''
def eachFile(filepath):
    pathDir =  os.listdir(filepath)
    f = []
    for allDir in pathDir:
        child = os.path.join('%s%s' % (filepath, allDir))
        f.append(child)
#        print (child) # .decode('gbk')是解决中文显示乱码问题
    return f

In [3]:
''' 读取SIC数据 '''
def ReadSIC(filename):
    
    file = Dataset(filename)
    lat = file.variables['lat'][:]
    lon = file.variables['lon'][:]
    sic = file.variables['ice_conc'][0][:]
    file.close()
    
    return sic,lat,lon    

In [4]:
''' 读取Pixel_Area数据 '''
def Read_PArea(filename):
    
    file = Dataset(filename)
    Area_sf = file.variables['area_sf'][:]  ## area scale factor
    Pixel_Area = Area_sf*100    ## Pixel_Area 单位:km^2
    file.close()
    
    return Pixel_Area      

In [5]:
''' 统计低值出现频次 '''   
def CountLow(sic,lat,Pixel_Area,lim1,lim2):
    """
    Input: sic,冰密集度日数据; 
           lat,纬度;
           Pixel_Area,格网面积：
           lim1,纬度下限;
           lim2,纬度上限
    Output: LCCA,低值面积/总面积*100%  
    """ 
    
    lat[lat<lim1] = np.nan  # 小于lim1的Lat置作nan;
    CA_sic = np.array(sic)
    CA_sic[lat != lat] = 999  #  纬度小于lim1区域的SIC记作999
    CA_sic[lat>lim2] = 999  # 纬度大于lim2区域的SIC记作999
#    a_hole = np.sum(Pix_Area[hdata != hdata])
#    print('未去除的空洞面积(nan)：%f' %a_hole)
    '''统计研究范围总面积 —— 非999的面积'''
    a_total = np.sum(Pixel_Area[CA_sic != 999])
#    print('总面积：%f' %a_total)  #  显示总面积
    '''统计低值面积 —— 小于75%且大于15%'''
    a_low = np.sum(Pixel_Area[CA_sic<75]) - np.sum(Pixel_Area[CA_sic<15])  
#    print('低值面积：%f' %a_low)  #  显示低值个数
    return a_low/a_total*100

In [6]:
''' main 函数  '''
if __name__=="__main__":
    start = time.clock()
    
    Geofilename = 'D:\\Arctic\\data\\GeoData\\r10km\\lmask_nh_stere_100.nc'
    Pixel_Area = Read_PArea(Geofilename)
    Year=['2017','2018','2019'] #'2008','2009','2010','2011','2012','2013','2014','2015','2016','2017'
#     FilePath = "D:\\Arctic\\data\\OSI SAF\\"  #测试路径
    for i in range(len(Year)):
        
        
        FilePath = "D:\\OSI SAF\\OSI-408\\"+Year[i]+"\\" 
        FileName = eachFile(FilePath)
        print(Year[i])
        
        file = open('OSI-AMSR2-SIC-LCCA-'+Year[i]+'-KD.txt','w')      
        for filename in FileName:
            date = filename[-15:-7]
            sic,lat,lon = ReadSIC(filename)
            
            lcca = CountLow(sic,lat,Pixel_Area,84,88.3)
            
            file.write('%10s %.2f' %(date,lcca))
            file.write('\n')

        file.close()
        
        end = time.clock()
        print('\n运行时间: %f' %(end-start))  # 显示运行时间
        
    print('success')

C:\Users\kathy\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until


2017


C:\Users\kathy\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in greater
  from ipykernel import kernelapp as app
C:\Users\kathy\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead



运行时间: 24.933479
2018

运行时间: 46.402689
2019

运行时间: 70.191822
success
